<a href="https://colab.research.google.com/github/mdjamina/machine_learning/blob/notebook/notebook/deft09_parlement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from pathlib import Path
import re
import pandas as pd

# Rapport d'Analyse de Données

## Corpus

## Prétraitement des Données

### Corpus d'apprentissage

In [13]:
train_file_path = '../deft09/Corpus d_apprentissage/deft09_parlement_appr_fr.xml'
tests_file_path = '../deft09/Corpus de test/deft09_parlement_test_fr.xml'
ref_file_path = '../deft09/Données de référence/deft09_parlement_ref_fr.txt'


In [14]:
Path(train_file_path).exists()

True

In [15]:
Path(tests_file_path).exists()

True

In [16]:
Path(tests_file_path).exists()

True

### Nétoyage des données

In [17]:

# Remplacer les caractères spéciaux
# cette permet de remplacer les caractères spéciaux


def replace_match(match: re.Match) -> str:

    char = match.group(0)
    replacements = {'<anonyme />':'ANONYME','’': "'",'´': "'",'`': "'",'‘': "'",'«': '"','»': '"','“': '"','”': '"','–': '-','—': '-','…': ' ',u'\0xa0': ' ',}

    return replacements[char]



#### corpus d'apprentissage

In [18]:
#Corriger le fichier train
pathfile = Path(train_file_path)
if not pathfile.exists():
    raise FileNotFoundError(f"{pathfile} not found")

pathfile_fix = pathfile.parent / f"{pathfile.stem}_fix{pathfile.suffix}"

with pathfile.open('r') as f:
    pattern = re.compile(r'<anonyme\s*/>|[’´`‘«»“”–—…]')
    content = re.sub(pattern, replace_match, f.read()).strip()
    with pathfile_fix.open('w') as f2:
        f2.write(content)
train_file_path = str(pathfile_fix)
print(train_file_path)

../deft09/Corpus d_apprentissage/deft09_parlement_appr_fr_fix.xml


#### corpus de test

In [19]:
#Corriger le fichier test
pathfile = Path(tests_file_path)
if not pathfile.exists():
    raise FileNotFoundError(f"{pathfile} not found")

pathfile_fix = pathfile.parent / f"{pathfile.stem}_fix{pathfile.suffix}"

with pathfile.open('r') as f:
    pattern = re.compile(r'<anonyme\s*/>|[’´`‘«»“”–—…]')
    content = re.sub(pattern, replace_match, f.read()).strip()
    with pathfile_fix.open('w') as f2:
        f2.write(content)
tests_file_path = str(pathfile_fix)
print(tests_file_path)


../deft09/Corpus de test/deft09_parlement_test_fr_fix.xml


### Chargement des données

In [34]:
from lxml import etree as et

#Parser le fichier xml
def parser(pathfile: str) -> list:

    # Parser le fichier XML avec lxml pour plus de performance
    tree = et.parse(pathfile)

    # Récupérer la racine du fichier XML
    root = tree.getroot()

    # Utiliser tqdm pour afficher la barre de progression
    for doc in root.findall('.//doc'):
        id_doc: str = doc.attrib['id']
        parti = None
        if doc.find('.//PARTI') is not None:
            parti = doc.find('.//PARTI').attrib['valeur']
        text = " ".join(p.text for p in doc.findall('.//p') if p.text)

        yield id_doc, parti, text


#### corpus d'apprentissage


In [36]:

data_train = pd.DataFrame(parser(str(train_file_path)), columns=['id', 'parti', 'text'])
data_train.head()

,id,parti,text
0,2_fr:1,Verts-ALE,"Monsieur le Président, j'ai toujours fait preu..."
1,2_fr:2,PPE-DE,"Madame la Présidente, chers collègues, à l'app..."
2,2_fr:3,Verts-ALE,"Je voudrais savoir si l'Union européenne, à la..."
3,2_fr:4,PSE,"Madame la Présidente, au nom des ANONYME de la..."
4,2_fr:5,PSE,"Monsieur le Président, chers collègues, Monsie..."


#### corpus de test

In [38]:
data_tests = pd.DataFrame([(id_doc,text) for id_doc,_,text in parser(str(tests_file_path))], columns=['index', 'text'])
data_tests = data_tests.set_index('index')

data_tests.head()

,text
index,
1,"Monsieur le Président, l'accès à des médias li..."
2,"Monsieur le Président, j'ai un point de vue di..."
3,"Monsieur le Président, Monsieur le Commissaire..."
4,Nous ne pouvons soutenir cette tentative d'éri...
5,"Monsieur le Président, ce court débat se dérou..."


#### données références

In [39]:
data_ref = pd.read_csv(ref_file_path, sep='\t', header=None, names=['index', 'parti'])
data_ref = data_ref.set_index('index')

data_ref.head()

,parti
index,
1,ELDR
2,GUE-NGL
3,PPE-DE
4,GUE-NGL
5,PPE-DE


### prétraitement des données

#### chargement du modèle spacy

#### prerequis

* installation des packages

```bash
pip install spacy
```    

* téléchargement du modèle

```bash
python -m spacy download fr_core_news_lg
```

In [68]:
import spacy

disable = ["vectors", "senter", "textcat"]

nlp = spacy.load('fr_core_news_lg', disable=disable)

nlp.pipe_names

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [85]:

from spacy.lang.fr.stop_words import STOP_WORDS

def lemmatization(text: str, stop_words=True, pos:list = None) -> list:

    #doc = nlp(text)

    document = []

    for token in nlp(text):
        is_stop= any([token.lemma_ in STOP_WORDS,
                       token.text in STOP_WORDS,
                       token.is_stop,
                       token.is_punct,
                       token.is_space,
                       token.is_digit
                       ])
        if ( stop_words and is_stop):
            continue
        if pos and token.pos_ not in pos:
            continue
        document.append(token.lemma_.lower())

    # return document, entities
    return " ".join(document)


In [87]:
lemmatization(data_train['text'][0],pos=['VERB'])[:100]

'encourager permettre éviter pouvoir soutenir répondre soulever exprimer soumettre voir retarder devo'

## Entraînement du Modèle

#### corpus d'apprentissage

In [74]:
y_train = data_train['parti'].to_numpy()
y_train[:5]

array(['Verts-ALE', 'PPE-DE', 'Verts-ALE', 'PSE', 'PSE'], dtype=object)

In [91]:
x_train = data_train['text'].to_numpy()
x_train[:5]

array(["monsieur président preuve scepticisme comportement moraliste policier encourager carrément délation permettre éviter cas fraude biais création instrument contrôle envahissant rapport pouvoir certainement groupe soutenir conviction rapport président napolitano modification règlement proposition décision répondre sérieusement préoccupation soulever récemment m. dell' alba dernier exprimer soumettre député contrôle excessif arbitraire voir aucun raison retarder davantage décision abus devoir produire parlement européen député disposer large possibilité défendre procédure interne transparent clair évidemment chose facile devoir confiance travail futur olaf devoir faciliter action manière pouvoir réagir crédible cas instrumentalisation pouvoir faire preuve efficacité agir répondre genre situation aujourd'hui mme theato m. bösch",
       'monsieur président cher collègue approche moment sombre année réjouissant évoquer été heure été vouloir attirer attention rituel réglage montre rép

#### Modèle de classification 

In [92]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [98]:
from joblib import Memory
mem = Memory(location="/tmp/cachedir", verbose=1)

In [93]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

tTfidVec = ('TfidfVectorizer', TfidfVectorizer(lowercase=False, max_df=0.95, min_df=10,sublinear_tf=True,preprocessor=lemmatization))

pipelines = {
    'LinearSVC': Pipeline([tTfidVec, ('LinearSVC', LinearSVC(max_iter=10000, dual="auto")) ])
    ,'MultinomialNB' :Pipeline([tTfidVec,('MultinomialNB', MultinomialNB()) ])
    , 'DecisionTreeClassifier': Pipeline([tTfidVec,('DecisionTreeClassifier', DecisionTreeClassifier()) ])

}

models = ((title,clf.fit(x_train, y_train)) for title,clf in pipelines.items())

#### entrainement du modèle

In [ ]:

# Sauvegarder les modèles
from joblib import dump, load
for title, model in models:
    dump(model, f'{title}.joblib')
    




## Évaluation du Modèle

In [ ]:
x_test = data_tests['text'].apply(lambda x : lemmatization(x,pos=pos)).to_numpy()
y_test = data_ref['parti'].to_numpy()

In [ ]:
from sklearn import metrics

for title, model in models:
    predictions = model.predict(x_test)
    print(title)
    print(metrics.classification_report(y_test, predictions, output_dict=False))
    print('confusion_matrix')
    print(metrics.confusion_matrix(y_test, predictions))
    print('accuracy_score')
    print(metrics.accuracy_score(y_test, predictions))
    print('precision_score')
    print(metrics.precision_score(y_test, predictions, average='macro'))
    print('recall_score')
    print(metrics.recall_score(y_test, predictions, average='macro'))
    print('f1_score')
    print(metrics.f1_score(y_test, predictions, average='macro'))
    

### affichage des résultats

sources: https://scikit-learn.org/stable/auto_examples/svm/plot_iris_svc.html

In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay
import matplotlib.pyplot as plt



# Set-up 2x2 grid for plotting.
fig, sub = plt.subplots(2, 2)
plt.subplots_adjust(wspace=0.4, hspace=0.4)

X0, X1 = x_test[:, 0], x_test[:, 1]

for clf, title, ax in zip(models.values(), models.keys(), sub.flatten()):
    disp = DecisionBoundaryDisplay.from_estimator(
        clf,
        x_test,
        response_method="predict",
        cmap=plt.cm.coolwarm,
        alpha=0.8,
        ax=ax,
        xlabel='discours',
        ylabel='parti',
    )
    ax.scatter(X0, X1, c=y_test, cmap=plt.cm.coolwarm, s=20, edgecolors="k")
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_title(title)


plt.show()